In [1]:
import torch

In [29]:
L = 64
d = 128

def chunk_attn(S_prev, Q, K, V):
    O_intra = torch.tril(Q @ K.T) @ V
    O_inter = Q @ S_prev
    O = O_inter + O_intra
    return O

def attn(Q, K, V):
    chunk_size = 16
    assert L%chunk_size==0
    n = L//chunk_size

    S_prev = torch.zeros(d, d)
    Q = Q.view(n, chunk_size, d)
    K = K.view(n, chunk_size, d)
    V = V.view(n, chunk_size, d)
    O = torch.zeros(n, chunk_size, d)
    
    for c in range(n):
        O[c] = chunk_attn(S_prev, Q[c], K[c], V[c])
        S_prev = S_prev + K[c].T @ V[c]

    O = O.view(L, d)
    return O

In [30]:
Q, K, V = torch.randn(L, d), torch.randn(L, d), torch.randn(L, d)

In [31]:
attn(Q, K, V).shape

torch.Size([64, 128])